In [1]:
#visualization part
#import matplotlib.pyplot as plt
#import seaborn as sns


#basic libs
import pandas as pd
import numpy as np
import random as rnd

from sklearn.model_selection import cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train_=pd.read_csv('../train_allcols.csv')
validate_=pd.read_csv('../validate_allcols.csv')
#test=pd.read_csv('../testwDSM.csv')

train_.shape, validate_.shape, #test.shape

((301619, 62), (148560, 62))

In [2]:
#train.describe()
train = train_.sample(10000)
validate = validate_.sample(3000)
train_.shape, #validate_.shape, validate.head(2)

((301619, 62),)

In [3]:
train = train.query('SUB2 != 1')
validate = validate.query('SUB2 != 1')

train = train[train.SUB2.isin([2,3,4,5,7,10])]
validate = validate[validate.SUB2.isin([2,3,4,7,10])]
print train.shape
print train['SUB2'].value_counts()

(4792, 62)
4     1681
2     1389
3      809
7      392
10     372
5      149
Name: SUB2, dtype: int64


In [4]:
# for sub2



X_train = train.drop(['SUB2'], axis=1)
Y_train = train["SUB2"]

from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif,chi2
#Selector_f = SelectPercentile(f_classif, percentile=25)
Selector_f = SelectKBest(f_classif, k=10)
Selector_f.fit(X_train,Y_train)

zipped = zip(X_train.columns.tolist(),Selector_f.scores_)
ans = sorted(zipped, key=lambda x: x[1])
for n,s in ans:    
    if 'FLG' in n: 
        pass
    else:
        print 'F-score: %3.2ft for feature %s' % (s,n)
                
#print X_train.columns.tolist()

F-score: 1.00t for feature VET
F-score: 1.06t for feature ARRESTS
F-score: 1.36t for feature PREG
F-score: 1.40t for feature DAYWAIT
F-score: 2.51t for feature ETHNIC
F-score: 2.82t for feature CBSA10
F-score: 2.89t for feature SUB3
F-score: 4.50t for feature PRIMINC
F-score: 4.56t for feature STFIPS
F-score: 5.18t for feature RACE
F-score: 5.27t for feature PRIMPAY
F-score: 5.66t for feature ROUTE3
F-score: 5.72t for feature DETNLF
F-score: 7.69t for feature LIVARAG
F-score: 8.13t for feature REGION
F-score: 8.25t for feature PSYPROB
F-score: 9.47t for feature EMPLOY
F-score: 9.68t for feature GENDER
F-score: 9.86t for feature YEAR
F-score: 10.02t for feature CASEID
F-score: 10.44t for feature MARSTAT
F-score: 10.77t for feature FRSTUSE1
F-score: 14.23t for feature FRSTUSE3
F-score: 14.37t for feature EDUC
F-score: 15.59t for feature HLTHINS
F-score: 17.53t for feature DIVISION
F-score: 18.86t for feature SERVSETA
F-score: 19.26t for feature DETCRIM
F-score: 20.49t for feature FREQ3
F

In [5]:
# get the sorted feature list
import sys
for n, s in ans:
    
    if 'FLG' in n: 
        pass
    else:
        sys.stdout.write('\'%s\',' % (n))


'VET','ARRESTS','PREG','DAYWAIT','ETHNIC','CBSA10','SUB3','PRIMINC','STFIPS','RACE','PRIMPAY','ROUTE3','DETNLF','LIVARAG','REGION','PSYPROB','EMPLOY','GENDER','YEAR','CASEID','MARSTAT','FRSTUSE1','FRSTUSE3','EDUC','HLTHINS','DIVISION','SERVSETA','DETCRIM','FREQ3','FREQ2','NUMSUBS','FREQ1','METHUSE','PSOURCE','ROUTE1','SUB1','NOPRIOR','AGE','DSMCRIT','IDU','ALCDRUG','ROUTE2','FRSTUSE2',

In [6]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['SUB2',  'ROUTE2', 'FREQ2', 'FRSTUSE2', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'NUMSUBS'
             ]
retain_list = ['EMPLOY','GENDER','FREQ1','YEAR','EDUC','PSYPROB','PSOURCE','SERVSETA','DETCRIM',
               'REGION','NOPRIOR','DIVISION','DSMCRIT','ROUTE1','SUB1','AGE','IDU','SUB3','ROUTE3',
               'FREQ3','FRSTUSE3','FREQ2','FRSTUSE2']
X_train = train[retain_list]
Y_train = train["SUB2"]
X_validate = validate[retain_list]
Y_validate = validate["SUB2"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((4792, 23), (1369, 23))

In [7]:
print X_train.columns.tolist()

['EMPLOY', 'GENDER', 'FREQ1', 'YEAR', 'EDUC', 'PSYPROB', 'PSOURCE', 'SERVSETA', 'DETCRIM', 'REGION', 'NOPRIOR', 'DIVISION', 'DSMCRIT', 'ROUTE1', 'SUB1', 'AGE', 'IDU', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'FREQ2', 'FRSTUSE2']


In [8]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((4792, 183), (1369, 183))

In [9]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_validate, Y_validate) * 100, 2)
print acc_log


68.66


In [10]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
#Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_validate, Y_validate) * 100, 2)
print acc_perceptron


60.34


/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [11]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


56.98


/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [12]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_validate, Y_validate) * 100, 2)
print acc_random_forest

#print cross_val_score(random_forest, X_validate, Y_validate)

66.47


In [13]:
from sklearn.ensemble import GradientBoostingClassifier,\
 AdaBoostClassifier, RandomForestClassifier
#Adaboost 
ada_boost = AdaBoostClassifier(random_state=1851)
ada_boost.fit(X_train, Y_train) 

acc_ada = round(ada_boost.score(X_validate, Y_validate) * 100, 2)
print acc_ada


61.87


In [14]:
# Stochastic Gradient Descent
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


64.57


In [15]:
# Linear SVC
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_validate, Y_validate) * 100, 2)
print acc_linear_svc


68.52


In [16]:
print 'predict-sub2-woflags-newsplit-sample20000'
models = pd.DataFrame({
    'Model': [ 'Logistic Regression', 
              'Random Forest',  'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
               'AdaBoost'],
    'Validate Set Score': [acc_log, 
              acc_random_forest,  acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_ada]
    })
print models.sort_values(by='Validate Set Score', ascending=False)


predict-sub2-woflags-newsplit-sample20000
                        Model  Validate Set Score
0         Logistic Regression               68.66
4                  Linear SVC               68.52
1               Random Forest               66.47
3  Stochastic Gradient Decent               64.57
5                    AdaBoost               61.87
2                  Perceptron               60.34
